In [1]:
# Load SQL extension
%load_ext sql

In [32]:
%%sql sqlite:///movies_data.db

-- Example SQL query
SELECT COUNT(*) FROM actors;

Done.


COUNT(*)
145


### Movies and Customers
1. **What are the often rented movies?**
2. **What are the frequent customers**
3. **Report movies with rating above avergage**
4. **Who are ths customers who gave low rating?**

#### Often reneted movies

In [3]:
%%sql
SELECT *
FROM movies
WhERE movies.movie_id IN  -- Select movie IDs from the inner query
    (SELECT movie_id
    FROM renting
    GROUP BY movie_id
    HAVING COUNT(*) > 5)
LIMIT 5;

 * sqlite:///movies_data.db
Done.


movie_id,title,genre,runtime,year_of_releas,renting_price
1,One Night at McCool's,Comedy,93,2001,2.09
2,Swordfish,Drama,99,2001,2.19
3,What Women Want,Comedy,127,2001,2.59
4,Training Day,Drama,122,2001,1.79
5,The Fellowship of the Ring,Science Fiction & Fantasy,178,2001,2.59


#### Frequent customers 

In [19]:
%%sql 
SELECT *
FROM customers
WHERE customer_id IN   -- Select all customers with more than 10 movie rentals
    (SELECT customer_id
    FROM renting
    GROUP BY customer_id
    HAVING COUNT(*) > 10);

 * sqlite:///movies_data.db
Done.


customer_id,name,country,gender,date_of_birth,date_account_start
21,Avelaine Corbeil,France,female,1986-03-17T00:00:00.000Z,2017-06-11T00:00:00.000Z
28,Sidney Généreux,France,male,1980-12-01T00:00:00.000Z,2017-02-04T00:00:00.000Z
49,Havasy Kristof,Hungary,male,1998-06-13T00:00:00.000Z,2017-01-18T00:00:00.000Z
92,Honorata Nowak,Poland,female,1986-05-02T00:00:00.000Z,2017-09-21T00:00:00.000Z
113,Lucy Centeno Barrios,Spain,female,1970-11-03T00:00:00.000Z,2017-06-13T00:00:00.000Z
114,Canela Gaona Lozano,Spain,female,1997-04-01T00:00:00.000Z,2017-02-14T00:00:00.000Z


#### Movies with rating above AVG ratings

In [29]:
%%sql
SELECT m.title AS movie_name-- Report the movie titles of all movies with average rating higher than the total average
FROM movies as m
WHERE m.movie_id IN
    (SELECT movie_id
     FROM renting
     GROUP BY movie_id
     HAVING AVG(rating) > 
        (SELECT AVG(rating)
         FROM renting))
LIMIT 5;

 * sqlite:///movies_data.db
Done.


movie_name
What Women Want
The Fellowship of the Ring
Harry Potter and the Philosopher's Stone
The Royal Tenenbaums
Waking Up in Reno


#### Customer who gave low ratings

In [30]:
%%sql
SELECT *
FROM customers AS c
WHERE 4 > -- Select all customers with a minimum rating smaller than 4 
    (SELECT MIN(rating)
    FROM renting AS r
    WHERE r.customer_id = c.customer_id);

 * sqlite:///movies_data.db
Done.


customer_id,name,country,gender,date_of_birth,date_account_start
28,Sidney Généreux,France,male,1980-12-01T00:00:00.000Z,2017-02-04T00:00:00.000Z
41,Zara Mitchell,Great Britan,female,1994-07-08T00:00:00.000Z,2017-06-12T00:00:00.000Z
86,Albin Jaworski,Poland,male,1984-05-01T00:00:00.000Z,2017-12-15T00:00:00.000Z
120,Robin J. Himes,USA,male,1988-11-30T00:00:00.000Z,2018-08-06T00:00:00.000Z


#### Customers with at least one rating

In [40]:
%%sql
-- Report the nationality and the number of actors for each nationality
SELECT a.nationality AS nationality, COUNT(*)  
FROM actors AS a
WHERE a.actor_id IN
    (SELECT ai.actor_id
     FROM actsin AS ai
     LEFT JOIN movies AS m
     ON m.movie_id = ai.movie_id
     WHERE m.genre = 'Comedy')
GROUP BY a.nationality LIMIT 5;


 * sqlite:///movies_data.db
Done.


nationality,COUNT(*)
British,3
Canada,1
Northern Ireland,1
South Africa,1
USA,22


### None-USA young actors
1. **What are ther actors who are either not from USA or born after 1990?**
2. **What are the actors who are not from USA nad born after 1990?**
2. **Dramas with high ratings?**

In [41]:
%%sql
SELECT name, 
       nationality, 
       year_of_birth
FROM actors
WHERE nationality <> 'USA'
UNION -- Select all actors who are not from the USA and all actors who are born after 1990
SELECT name, 
       nationality, 
       year_of_birth
FROM actors
WHERE year_of_birth > 1990
LIMIT 5;

 * sqlite:///movies_data.db
Done.


name,nationality,year_of_birth
Abbie Cornish,Australia,1982.0
Andrea Riseborough,British,1981.0
Annasophia Robb,USA,1993.0
Anthony Hopkins,British,1937.0
Barkhad Abdi,Somalia,1985.0


#### Dramas with high ratings 

In [42]:
%%sql
SELECT *
FROM movies
WHERE movie_id IN -- Select all movies of genre drama with average rating higher than 9
   (SELECT movie_id
    FROM movies
    WHERE genre = 'Drama'
    INTERSECT
    SELECT movie_id
    FROM renting
    GROUP BY movie_id
    HAVING AVG(rating)>9);

 * sqlite:///movies_data.db
Done.


movie_id,title,genre,runtime,year_of_releas,renting_price
42,No Country for Old Men,Drama,122,2007,1.49
